In [1]:
import pandas as pd 
import numpy as np
from fuzzywuzzy import fuzz
from tqdm import tqdm
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re
from collections import Counter

/Users/jerry/opt/anaconda3/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
stanford_cleand = pd.read_csv('drive-download-20220516T014219Z-001/final_reformatted_stanford.csv')
patent_data = pd.read_csv('drive-download-20220516T014219Z-001/complete_grouped.csv') 
public = pd.read_csv('drive-download-20220516T014219Z-001/final_public.csv')

In [3]:
def remove_chars_and_lower(string, chars, words):
    '''
    Function to remove quotation marks from a string, since there seems to be some issues of excess quotes
    placed into the company names
    '''
    string = ''.join(i for i in string if i not in chars)
    
    string_words = string.split()
    result_words  = [word for word in string_words if word.lower() not in words]
    result = ' '.join(result_words)
    
    return result.lower()

drop_chars = ['"', '.', '%']

drop_words = ['.com', 'llc', 'incorporated', 'inc', 'co', 'holdings', 'ho', 'corporation', 'corp', 
              'company', 'hldg', 'ent', 'ltd', 'association', 'foundation', 'club', 'fund', 'the', 'and', 'of', 
              'services', 'enterprises']

# companylist=['bottomline technologies inc', 'arrow international llc', 'american home products corp',
#             'aladdin knowledge systems ltd', "alan b clement"]
# # for i in companylist:
# #     print(re.sub(r"\W(inc|llc|corp|ltd|co|ltd)\b", "", i))
# comp_list = pd.DataFrame(companylist, columns = ['test'])   
# comp_list.test= comp_list.test.apply(lambda x: re.sub(r"\W(inc|llc|corp|ltd|co|ltd|lp)\b", "", x))

In [4]:
# trevor cleaning technique
public['Company'] = public['Company'].apply(lambda x: x.lower())
public['Company'] = public['Company'].apply(lambda x: remove_chars_and_lower(x, drop_chars, drop_words))

In [5]:
# some low key data preprocesing

# lowercase patent_data
public.Company = public.Company.str.lower()

# remove all commas
public.Company = public.Company.apply(lambda x : x.replace(',',''))


# remove some specifics endings 
public.Company = public.Company.apply(lambda x: re.sub(r"\W(inc|llc|corp|ltd|co|ltd|lp)\b", "", x))

# stripping leading and trailing white spaces; and apply lower case to all names
public.Company = public.Company.replace(r"^ +| +$", r"", regex=True)




In [6]:
# some low key data preprocesing

# lowercase patent_data
stanford_cleand.Infringer = stanford_cleand.Infringer.str.lower()
patent_data.correspondence_name = patent_data.correspondence_name.str.lower()

# remove all commas
stanford_cleand.Infringer = stanford_cleand.Infringer.apply(lambda x : x.replace(',',''))
patent_data.correspondence_name = patent_data.correspondence_name.apply(lambda x : x.replace(',',''))


# remove some specifics endings 
stanford_cleand.Infringer = stanford_cleand.Infringer.apply(lambda x: re.sub(r"\W(inc|llc|corp|ltd|co|ltd|lp)\b", "", x))
patent_data.correspondence_name = patent_data.correspondence_name.apply(lambda x: re.sub(r"\W(inc|llc|corp|ltd|co|ltd|lp)\b", "", x))

# stripping leading and trailing white spaces; and apply lower case to all names
stanford_cleand.Infringer = stanford_cleand.Infringer.replace(r"^ +| +$", r"", regex=True)
patent_data.correspondence_name = patent_data.correspondence_name.replace(r"^ +| +$", r"", regex=True)

In [7]:
print(stanford_cleand.shape)
print(patent_data.shape)
print(public.shape)

(1605274, 9)
(8478668, 5)
(611314, 13)


In [8]:
lit_df = patent_data.merge(stanford_cleand, how='left', left_on=['correspondence_name', 'Year'], 
                                right_on=['Infringer', 'Year'])

lit_df.dropna(subset=['correspondence_name'], inplace=True)
print(len(lit_df['correspondence_name'].unique()))

364318


In [9]:
lit_df.Infringer.unique()

array([nan, '10x genomics', '12 sigma technologies', ..., 'zte (usa)',
       'zymogenetics', 'zyvex'], dtype=object)

In [10]:
print(len(lit_df.dropna(subset=['Infringer'])['correspondence_name'].unique()))

3280


In [11]:
lit_df.dropna(subset=['Infringer'])['correspondence_name'].unique()

array(['10x genomics', '12 sigma technologies', '2go products', ...,
       'zte (usa)', 'zymogenetics', 'zyvex'], dtype=object)

# Merged the seoncd 

In [12]:
full_lit_df = lit_df.merge(public, how='left', left_on=['correspondence_name', 'Year'], 
                                right_on=['Company', 'Year'])
print(len(full_lit_df['correspondence_name'].unique()))

364318


In [13]:
print(len(full_lit_df['Company'].unique()))

1059
